## API 구분


- [네이버](https://developers.naver.com/docs/common/openapiguide/)
    - 통합 검색어 트렌드 API
    - 쇼핑인사이트 API

- [클라우드 API](https://api.ncloud-docs.com/docs/api-overview)
    - Geocoding
    - Reverse Geocoding
    - Static Map

In [1]:
from config import config

# 네이버 파파고 API
papago_id = config['naver']['papago']['client_id']
papago_pw = config['naver']['papago']['client_secret']

# 한글 인명-로마자 변환 API
romanization_id = config['naver']['romanization']['client_id']
romanization_pw = config['naver']['romanization']['client_secret']

# 네이버 통합 검색어 API
search_id = config['naver']['search']['client_id']
search_pw = config['naver']['search']['client_secret']

# 네이버 단축 URL API
shorturl_id = config['naver']['shorturl']['client_id']
shorturl_pw = config['naver']['shorturl']['client_secret']

# 네이버 쇼핑인사이트 API
shopping_id = config['naver']['shopping']['client_id']
shopping_pw = config['naver']['shopping']['client_secret']

# 네이버 클라우드 플랫폼 API
cloud_id = config['cloud']['client_id']
cloud_pw = config['cloud']['client_secret']

In [2]:
import json
import datetime
import requests
import pandas as pd

## MAP

In [28]:
class Map:
    """
    네이버 지도 API 클래스
    """
    def _init_(self):
        pass

    def transit_directions_point_to_point(self, **kwargs):
        """
        네이버 지도 길찾기 대중교통 API
        """
        url = "https://map.naver.com/v5/api/transit/directions/point-to-point?"
        params = {
            "crs": "EPSG:4326",
            "mode": "TIME",
            "lang": "ko",
            "includeDetailOperation": "true",
        }
        kwargs.update(params)
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url)
        if res.status_code == 200:
            return res.json()
        else:
            return res

### 지도 길찾기 대중교통

In [29]:
# 네이버 지도 API 인스턴스
naver_map = Map()

# 파라미터
sx, sy = 127.019534, 37.51608
ex, ey = 127.111203, 37.39475
start = f"{sx},{sy}"
goal = f"{ex},{ey}"
today = datetime.date.today()
departureTime = (today + datetime.timedelta(days=-today.weekday(), weeks=1)).strftime("%Y-%m-%dT08:00:00")

# API 호출
data = naver_map.transit_directions_point_to_point(start=start,
                                         goal=goal,
                                         departureTime=departureTime)
data

{'status': 'CITY',
 'context': {'engineVersion': '1.3.0:73cd4304-c049-4598-ab0c-859eaea8edc8',
  'engineUrl': None,
  'start': '127.019534,37.51608,,,,',
  'goal': '127.111203,37.39475,,,,',
  'departureTime': '2022-09-05T08:00:00',
  'currentDateTime': '2022-09-01T23:19:40',
  'serviceDay': {'id': 1, 'name': '평일'},
  'recommendDirectionsType': 'CITY',
  'uuid': None},
 'paths': [{'idx': 1,
   'mode': 'TIME',
   'type': 'SUBWAY',
   'optimizationMethod': 'RECOMMEND',
   'labels': ['최적'],
   'departureTime': '2022-09-05T08:03:20',
   'arrivalTime': '2022-09-05T08:21:43',
   'alarmDepartureTime': None,
   'alarmArrivalTime': None,
   'duration': 18,
   'intercityDuration': 0,
   'walkingDuration': 0,
   'waitingDuration': 0,
   'distance': 16500,
   'shutdown': False,
   'fare': 2950,
   'fares': [{'routes': [[{'id': 109,
        'name': '신분당선',
        'longName': '수도권 신분당선(신사-광교)',
        'type': {'id': 109,
         'name': '신분당선',
         'shortName': None,
         'longName': Non

## NAVER

In [5]:
class Naver:
    """
    네이버 OPEN API 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        API 인증 정보 초기화
        """
        self.headers = {
            "X-Naver-Client-Id": client_id,
            "X-Naver-Client-Secret": client_secret,
            "Content-Type": "application/json",
        }

    def datalab_search(self, **kwargs):
        """
        네이버 통합 검색어 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/search/search.md
        """
        url = "https://openapi.naver.com/v1/datalab/search"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            num = len(data)
            df = pd.DataFrame()
            for i in range(num):
                sub = pd.DataFrame(data[i]['data'])
                sub['title'] = data[i]['title']
                sub = sub[['title', 'period', 'ratio']]
                df = pd.concat([df, sub], axis=0, ignore_index=True)
            pivot = pd.pivot(df, index='period', columns='title',
                             values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_categories(self, **kwargs):
        """
        쇼핑인사이트 분야별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/categories"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            num = len(data)
            df = pd.DataFrame()
            for i in range(num):
                sub = pd.DataFrame(data[i]['data'])
                sub['title'] = data[i]['title']
                sub = sub[['title', 'period', 'ratio']]
                df = pd.concat([df, sub], axis=0, ignore_index=True)
            pivot = pd.pivot(df, index='period', columns='title',
                             values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_device(self, **kwargs):
        """
        쇼핑인사이트 분야 내 기기별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/device"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_gender(self, **kwargs):
        """
        쇼핑인사이트 분야 내 성별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/gender"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_age(self, **kwargs):
        """
        쇼핑인사이트 분야 내 연령별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/age"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keywords(self, **kwargs):
        """
        쇼핑인사이트 키워드별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keywords"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            num = len(data)
            df = pd.DataFrame()
            for i in range(num):
                sub = pd.DataFrame(data[i]['data'])
                sub['title'] = data[i]['title']
                sub = sub[['title', 'period', 'ratio']]
                df = pd.concat([df, sub], axis=0, ignore_index=True)
            pivot = pd.pivot(df, index='period', columns='title',
                             values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keyword_device(self, **kwargs):
        """
        쇼핑인사이트 키워드 기기별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/device"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keyword_gender(self, **kwargs):
        """
        쇼핑인사이트 키워드 성별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/gender"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keyword_age(self, **kwargs):
        """
        쇼핑인사이트 키워드 연령별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/age"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def util_shorturl(self, url):
        """
        단축 URL 요청(JSON)
        - API 레퍼런스: https://developers.naver.com/docs/utils/shortenurl/
        """
        req_url = f"https://openapi.naver.com/v1/util/shorturl?url={url}"
        res = requests.get(req_url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res
        
    def papago_n2mt(self, **kwargs):
        """
        Papago 번역
        - API 레퍼런스: https://developers.naver.com/docs/papago/papago-nmt-api-reference.md
        """
        url = "https://openapi.naver.com/v1/papago/n2mt"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            return res.json()['message']['result']['translatedText']
        else:
            return res
        
    def krdict_romanization(self, **kwargs):
        """
        한글 인명-로마자 변환
        - API 레퍼런스: https://developers.naver.com/docs/papago/papago-romanization-api-reference.md
        """
        url = "https://openapi.naver.com/v1/krdict/romanization?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['aResult'][0]['aItems'])
        else:
            return res

### 통합 검색어 트렌드 API

In [6]:
# 네이버 API 인스턴스
naver = Naver(search_id, search_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
keywordGroups = [
    {
        "groupName": "미국",
        "keywords": ["연준", "FRB", "FED", "FOMC"]
    },

    {
        "groupName": "한국",
        "keywords": ["한국은행", "금융통화위원회"]
    }
]


# 요청
df = naver.datalab_search(startDate=startDate,
                          endDate=endDate,
                          timeUnit=timeUnit,
                          keywordGroups=keywordGroups)
df

,날짜,미국,한국
0,2022-01-01,0.98301,0.59953
1,2022-01-02,1.19120,0.80558
2,2022-01-03,1.97032,1.79861
3,2022-01-04,2.11269,1.82866
4,2022-01-05,5.29640,1.88733
...,...,...,...
237,2022-08-26,5.09107,2.20284
238,2022-08-27,4.42643,0.96441
239,2022-08-28,2.43321,0.89072
240,2022-08-29,3.78110,2.29155


### Shopping Insight API 1

In [7]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = [
    {
        "name": "카테고리1",
        "param": ["50000000"],
    },
    {
        "name": "카테고리2",
        "param": ["50000002"],
    },
]

# 요청
df = naver.datalab_shopping_categories(startDate=startDate,
                                       endDate=endDate,
                                       timeUnit=timeUnit,
                                       category=category)
df

,날짜,카테고리1,카테고리2
0,2022-01-01,76.73259,26.60887
1,2022-01-02,90.15836,31.41484
2,2022-01-03,79.15109,31.64302
3,2022-01-04,78.83863,31.76123
4,2022-01-05,78.73141,31.14685
...,...,...,...
237,2022-08-26,52.63769,31.84794
238,2022-08-27,59.23958,34.21125
239,2022-08-28,69.51918,37.97973
240,2022-08-29,68.35153,41.28070


### Shopping Insight API 2

In [8]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000002"

# 요청
df = naver.datalab_shopping_category_device(startDate=startDate,
                                            endDate=endDate,
                                            timeUnit=timeUnit,
                                            category=category)
df

,날짜,mo,pc
0,2022-01-01,67.01982,9.45582
1,2022-01-02,79.64081,10.64751
2,2022-01-03,75.41365,15.53046
3,2022-01-04,74.95823,16.32563
4,2022-01-05,73.27568,16.24241
...,...,...,...
237,2022-08-26,78.72732,12.80574
238,2022-08-27,87.39495,10.93043
239,2022-08-28,98.34492,10.81131
240,2022-08-29,99.60289,19.04054


### Shopping Insight API 3

In [9]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000002"

# 요청
df = naver.datalab_shopping_category_gender(startDate=startDate,
                                            endDate=endDate,
                                            timeUnit=timeUnit,
                                            category=category)
df

,날짜,f,m
0,2022-01-01,70.62519,15.94354
1,2022-01-02,87.33908,19.89183
2,2022-01-03,84.87191,20.35160
3,2022-01-04,84.57151,20.07884
4,2022-01-05,83.86027,19.12518
...,...,...,...
237,2022-08-26,61.90122,14.80623
238,2022-08-27,62.25493,13.76774
239,2022-08-28,70.99948,16.35938
240,2022-08-29,72.22320,17.70587


### Shopping Insight API 4

In [10]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000002"

# 요청
df = naver.datalab_shopping_category_age(startDate=startDate,
                                         endDate=endDate,
                                         timeUnit=timeUnit,
                                         category=category)
df

,날짜,10,20,30,40,50,60
0,2022-01-01,6.08997,43.16787,73.05869,70.93650,33.71510,9.45276
1,2022-01-02,7.14473,55.24364,92.60173,87.12545,41.64197,11.41368
2,2022-01-03,6.21985,56.44833,93.61679,83.44995,38.66883,11.03897
3,2022-01-04,6.31230,55.43327,91.95969,83.06275,39.73966,11.45698
4,2022-01-05,6.33159,54.46415,89.81046,82.27475,39.61375,11.09399
...,...,...,...,...,...,...,...
237,2022-08-26,4.02941,40.17847,60.31062,60.91826,30.07363,8.68235
238,2022-08-27,5.36549,38.65635,57.04160,60.71955,30.75652,8.61504
239,2022-08-28,6.31646,43.47433,69.36201,70.67938,34.37832,9.14743
240,2022-08-29,4.81193,46.23554,73.90643,72.68152,34.09530,9.58416


### Shopping Insight API 5

In [11]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = [
    {
        "name": "키워드1",
        "param": ["정장"],
    },
    {
        "name": "키워드2",
        "param": ["비즈니스 캐주얼"],
    },
]

# 요청
df = naver.datalab_shopping_category_keywords(startDate=startDate,
                                              endDate=endDate,
                                              timeUnit=timeUnit,
                                              category=category,
                                              keyword=keyword)
df

,날짜,키워드1,키워드2
0,2022-01-01,33.62487,0.68027
1,2022-01-02,41.01068,0.38872
2,2022-01-03,38.77551,0.29154
3,2022-01-04,43.05150,0.29154
4,2022-01-05,42.37123,0.48590
...,...,...,...
237,2022-08-26,27.79397,0.38872
238,2022-08-27,21.57434,0.29154
239,2022-08-28,32.75024,0.97181
240,2022-08-29,43.24586,0.19436


### Shopping Insight API 6

In [12]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = "정장"

# 요청
df = naver.datalab_shopping_category_keyword_device(startDate=startDate,
                                                    endDate=endDate,
                                                    timeUnit=timeUnit,
                                                    category=category,
                                                    keyword=keyword)
df

,날짜,mo,pc
0,2022-01-01,29.14798,9.64125
1,2022-01-02,38.67713,8.63228
2,2022-01-03,37.21973,7.51121
3,2022-01-04,41.14349,8.52017
4,2022-01-05,42.04035,6.83856
...,...,...,...
237,2022-08-26,24.43946,7.62331
238,2022-08-27,19.05829,5.82959
239,2022-08-28,29.82062,7.95964
240,2022-08-29,33.07174,16.81614


### Shopping Insight API 7

In [13]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = "정장"

# 요청
df = naver.datalab_shopping_category_keyword_gender(startDate=startDate,
                                                    endDate=endDate,
                                                    timeUnit=timeUnit,
                                                    category=category,
                                                    keyword=keyword)
df

,날짜,f,m
0,2022-01-01,11.94029,38.43283
1,2022-01-02,8.76865,58.95522
2,2022-01-03,6.71641,58.20895
3,2022-01-04,9.14179,60.26119
4,2022-01-05,4.47761,71.08208
...,...,...,...
237,2022-08-26,5.41044,36.38059
238,2022-08-27,12.87313,23.32089
239,2022-08-28,6.52985,45.70895
240,2022-08-29,8.76865,64.36567


### Shopping Insight API 8

In [14]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = "정장"

# 요청
df = naver.datalab_shopping_category_keyword_age(startDate=startDate,
                                                 endDate=endDate,
                                                 timeUnit=timeUnit,
                                                 category=category,
                                                 keyword=keyword)
df

,날짜,10,20,30,40,50,60
0,2022-01-01,0.31545,14.51104,33.75394,34.70031,0.94637,0.63091
1,2022-01-02,4.41640,18.92744,44.16403,24.92113,16.08832,3.47003
2,2022-01-03,1.89274,15.77287,38.48580,35.33123,17.66561,0.31545
3,2022-01-04,NaN,23.34384,28.07570,47.94952,9.77917,NaN
4,2022-01-05,0.31545,16.08832,56.46687,49.21135,4.41640,1.26182
...,...,...,...,...,...,...,...
237,2022-08-26,2.52365,10.72555,32.49211,21.76656,2.83911,0.31545
238,2022-08-27,0.94637,6.30914,22.39747,25.23659,4.10094,NaN
239,2022-08-28,1.89274,11.67192,50.15772,20.82018,3.47003,NaN
240,2022-08-29,1.26182,20.18927,42.90220,50.15772,8.83280,0.31545


### 단축 URL API

In [15]:
# 네이버 API 인스턴스
naver = Naver(shorturl_id, shorturl_pw)

# 파라미터
url = "https://wooiljeong.github.io"

# 단축 URL API 호출
res = naver.util_shorturl(url)
res

{'result': {'url': 'https://me2.do/FniJlxxK',
  'hash': 'FniJlxxK',
  'orgUrl': 'https://wooiljeong.github.io'},
 'message': 'ok',
 'code': '200'}

### Papago 번역 API

In [16]:
# 네이버 API 인스턴스
naver = Naver(papago_id, papago_pw)

# 파라미터
source = "ko"
target = "en"
text = "파이썬으로 파파고 번역 API를 호출해봅니다."

# 단축 URL API 호출
res = naver.papago_n2mt(source = source, 
                        target = target, 
                        text = text)
res

'Try calling the Papago translation API with Python.'

### 한글 인명 로마자 변환 API

In [17]:
# 네이버 API 인스턴스
naver = Naver(romanization_id, romanization_pw)

# 파라미터
query = "홍길동"

# 단축 URL API 호출
df = naver.krdict_romanization(query=query)
df

,name,score
0,Hong Gildong,99
1,Hong Kildong,96
2,Hong Gildoung,21
3,Hong Kildoung,20


## NAVER CLOUD PLATFORM

In [18]:
class NaverCloudPlatform:
    """
    네이버 클라우드 플랫폼 OPEN API 클래스
    """
    def __init__(self, client_id, client_secret):
        """
        API 인증 정보 초기화
        """
        self.headers = {
            "X-NCP-APIGW-API-KEY-ID": client_id,
            "X-NCP-APIGW-API-KEY": client_secret,
        }
        
    def geocoding(self, query):
        """
        지오코딩 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsgeocoding
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={query}"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

    def reverse_geocoding(self, lon, lat):
        """
        리버스 지오코딩 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsreversegeocoding
        """
        coords = f"{lon},{lat}"
        url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords}&output=json&orders=addr"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res
        
    def directions5(self, **kwargs):
        """
        Directions5 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsdirections
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

    def directions15(self, **kwargs):
        """
        Directions15 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsdirections15
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

### Directions5

In [19]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 파라미터
start = "127.1115061,37.3864539"
goal = "127.01953476,37.5160802"

# 지오코딩 요청
res = ncp.directions5(start=start, goal=goal)
res

{'code': 0,
 'message': '길찾기를 성공하였습니다.',
 'currentDateTime': '2022-09-01T23:15:42',
 'route': {'traoptimal': [{'summary': {'start': {'location': [127.111505,
       37.3864536]},
     'goal': {'location': [127.0195345, 37.51608], 'dir': 2},
     'distance': 19001,
     'duration': 1364823,
     'etaServiceType': 1,
     'departureTime': '2022-09-01T23:15:42',
     'bbox': [[127.0092506, 37.3863436], [127.1130969, 37.5160872]],
     'tollFare': 1000,
     'taxiFare': 18430,
     'fuelPrice': 2364},
    'path': [[127.1115011, 37.3863436],
     [127.1113757, 37.3863459],
     [127.1113214, 37.3863457],
     [127.1113223, 37.3864025],
     [127.1113213, 37.3865918],
     [127.1113229, 37.3869451],
     [127.1113227, 37.3869785],
     [127.1113209, 37.3873291],
     [127.1113207, 37.3873625],
     [127.1113202, 37.3874562],
     [127.1113186, 37.3877735],
     [127.1113184, 37.3878141],
     [127.1113193, 37.3878519],
     [127.1113189, 37.387933],
     [127.1114544, 37.3879344],
     [127.

### Directions15

In [20]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 파라미터
start = "127.1115061,37.3864539"
goal = "127.01953476,37.5160802"

# 지오코딩 요청
res = ncp.directions15(start=start, goal=goal)
res

{'code': 0,
 'message': '길찾기를 성공하였습니다.',
 'currentDateTime': '2022-09-01T23:15:42',
 'route': {'traoptimal': [{'summary': {'start': {'location': [127.111505,
       37.3864536]},
     'goal': {'location': [127.0195345, 37.51608], 'dir': 2},
     'distance': 19001,
     'duration': 1364675,
     'etaServiceType': 1,
     'departureTime': '2022-09-01T23:15:42',
     'bbox': [[127.0092506, 37.3863436], [127.1130969, 37.5160872]],
     'tollFare': 1000,
     'taxiFare': 18430,
     'fuelPrice': 2364},
    'path': [[127.1115011, 37.3863436],
     [127.1113757, 37.3863459],
     [127.1113214, 37.3863457],
     [127.1113223, 37.3864025],
     [127.1113213, 37.3865918],
     [127.1113229, 37.3869451],
     [127.1113227, 37.3869785],
     [127.1113209, 37.3873291],
     [127.1113207, 37.3873625],
     [127.1113202, 37.3874562],
     [127.1113186, 37.3877735],
     [127.1113184, 37.3878141],
     [127.1113193, 37.3878519],
     [127.1113189, 37.387933],
     [127.1114544, 37.3879344],
     [127.

### Geocoding

In [21]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 주소
query = "해밀동 산8"

# 지오코딩 요청
res = ncp.geocoding(query)
res

{'status': 'OK',
 'meta': {'totalCount': 1, 'page': 1, 'count': 1},
 'addresses': [{'roadAddress': '세종특별자치시 해밀3로 32 해밀마을1단지',
   'jibunAddress': '세종특별자치시 해밀동 산8 해밀마을1단지',
   'englishAddress': '32, Haemil 3-ro, Sejong-si, Republic of Korea',
   'addressElements': [{'types': ['SIDO'],
     'longName': '세종특별자치시',
     'shortName': '세종특별자치시',
     'code': ''},
    {'types': ['SIGUGUN'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['DONGMYUN'], 'longName': '해밀동', 'shortName': '해밀동', 'code': ''},
    {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['ROAD_NAME'],
     'longName': '해밀3로',
     'shortName': '해밀3로',
     'code': ''},
    {'types': ['BUILDING_NUMBER'],
     'longName': '32',
     'shortName': '32',
     'code': ''},
    {'types': ['BUILDING_NAME'],
     'longName': '해밀마을1단지',
     'shortName': '해밀마을1단지',
     'code': ''},
    {'types': ['LAND_NUMBER'],
     'longName': '산8',
     'shortName': '산8',
     'code': ''},
    {'types': ['PO

### Reverse Geocoding

In [22]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 경도, 위도
lon, lat = 127.228954, 37.376474

# 지오코딩 요청
res = ncp.reverse_geocoding(lon, lat)
res

{'status': {'code': 0, 'name': 'ok', 'message': 'done'},
 'results': [{'name': 'addr',
   'code': {'id': '4161025021', 'type': 'L', 'mappingId': '0261025021'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '경기도',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 127.550802,
       'y': 37.4363177}},
     'alias': '경기'},
    'area2': {'name': '광주시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 127.2550476,
       'y': 37.4294306}}},
    'area3': {'name': '오포읍',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 127.2285341,
       'y': 37.3663057}}},
    'area4': {'name': '고산리',
     'coords': {'center': {'crs': 'EPSG:4326', 'x': 127.2234, 'y': 37.3742}}}},
   'land': {'type': '1',
    'number1': '353',
    'number2': '',
    'addition0': {'type': '', 'value': ''},
    'addition1': {'type': '', 'value': ''},
    'addition2': {'type': '', 'value': ''},
    'addition3': {'type': '', '